In [1]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
import rispy
import pandas as pd
from tqdm import tqdm
from itertools import chain

# Path to your RIS file
ris_file = 'TAS_200 records.ris'

# Read the RIS file with rispy
with open(ris_file, 'r') as file:
    records = rispy.load(file)

# Convert to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
df.head(2)


,type_of_reference,title,authors,abstract,date,year,secondary_title,doi,volume,number,alternate_title1,start_page,end_page,url,accession_number
0,JOUR,Screening for Cognitive Impairment in Older Ad...,"[Patnode, Carrie D., Perdue, Leslie A., Rossom...",OBJECTIVE: We conducted this systematic review...,2020///,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JOUR,Interventions to Prevent Falls in Community-Dw...,"[Guirguis-Blake, Janelle M., Michael, Yvonne L...",OBJECTIVE: We conducted this systematic review...,2018///,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
def evaluate_items_with_prompt(item_list, prompt_template, api_key=""):
    """
    Evaluates a list of items against a prompt to determine if they are True or False.

    Args:
        item_list (list): List of items to evaluate.
        prompt_template (str): The prompt template to use for evaluation.
        api_key (str): OpenAI API key.

    Returns:
        dict: A dictionary of items and their True/False evaluation.
    """
    # Initialize the ChatOpenAI model with GPT-4
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        openai_api_key=api_key
    )

    # Create the prompt
    prompt = PromptTemplate(input_variables=["item"], template=prompt_template)
    
    # Create the chain
    chain = prompt | llm

    # Evaluate each item
    results = {}
    for item in tqdm(item_list):
        response = chain.invoke({"item": item}).content.strip().lower()
        # Normalize to True/False
        results[item] = response
#         results[item] = response in ["true", "yes"]
    
    return results



In [61]:
items = df["title"].astype(str) + "\n" + df["abstract"].astype(str)
items = items.tolist()

In [62]:
# Define the prompt template
prompt = """
Types of studies: 
We will include individually- and cluster-randomized controlled trials (RCTs). \
We will include both parallel and cross-over trials, where only data from the first \
intervention phase, prior to cross-over, will be used.

Examples:
1- Randomized controlled and crossover trials: 
      Implementation of a multicomponent intervention to prevent physical restraints in nursing homes (IMPRINT): A pragmatic cluster randomized controlled trial
      Economic Evaluation of Exercise or Cognitive and Social Enrichment Activities for Improved Cognition After Stroke
      Digital Intervention Promoting Physical Activity in People Newly Diagnosed with Parkinson’s Disease: Feasibility and Acceptability of the Knowledge, Exercise-Efficacy and Participation (KEEP) Intervention
      Rivastigmine Transdermal Patch and Physical Exercises for Alzheimer's Disease: A Randomized Clinical Trial
      The effects of probiotic supplements on oxidative stress and inflammation in subjects with mild and moderate Alzheimer’s disease: a randomized, double-blind, placebo-controlled study
      A 3 month nutrition and exercise program improved hallux strength among senior daycare center users in Korea: a cluster randomized controlled trial
      Effects of dual-task resistance exercise on cognition, mood, depression, functional fitness, and activities of daily living in older adults with cognitive impairment: a single-blinded, randomized controlled trial
      Effect of three modalities on emergence agitation among post-traumatic stress disorder patients undergoing laparoscopy: a randomized controlled study
      Improving Sleep Quality, Daytime Sleepiness, and Cognitive Function in Patients with Dementia by Therapeutic Exercise and NESA Neuromodulation: A Multicenter Clinical Trial
      Tango-therapy vs physical exercise in older people with dementia; a randomized controlled trial
      
2-Other study designs: 
      The Role of Physical Activity in Adjunctive Nursing Management of Neuro-Degenerative Diseases among Older Adults: A Systematic Review of Interventional Studies (study design: systematic review)
      Effects of exercise therapy on global cognitive function and, depression in older adults with mild cognitive impairment: A systematic review and meta-analysis (study design: systematic review with meta-analysis)
      Association of Long-Term Diet Quality with Hippocampal Volume: Longitudinal Cohort Study (study design: longitudinal cohort study) 
      Long-term cumulative depressive symptom burden and risk of cognitive decline and dementia among very old women (study design: longitudinal cohort study) 
      Experiences of Participation in a Multimodal Preventive Trial MIND-ADMINI Among Persons with Prodromal Alzheimer’s Disease: A Qualitative Study (study design: qualitative study) 
      Interventions to Prevent Falls in Community-Dwelling Older Adults: A Systematic Review for the U.S. Preventive Services Task Force (study design: systematic review)
      Effect of Dancing Interventions on Depression and Anxiety Symptoms in Older Adults: A Systematic Review and Meta-Analysis (study design: systematic review with meta-analysis)
      Dose-response relationship of high-intensity training on global cognition in older adults with mild cognitive impairment or dementia: a systematic review with meta-analysis - the ACHIEVE-Study (study design: systematic review with meta-analysis)
      Evaluation of a frailty index based on a comprehensive geriatric assessment in a population based study of elderly Canadians (study design: prospective cohort study)
      Associations of socioeconomic status and healthy lifestyle with incident dementia and cognitive decline: two prospective cohort studies (study design: prospective cohort study)
      
Based on the study type description mentioned before and the examples, \
determine the study design type in the following text which are the title and abstract \
of a scientific paper: '{item}'. 

Respond with only the study type you find without any punctuation.
"""

In [63]:
# Run the evaluation
results = evaluate_items_with_prompt(items, prompt)

# Display the results
i = 0
for item, result in results.items():
    print(f"Statement: 'record_{i}' - Result: {result}")
    i += 1

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [07:56<00:00,  2.38s/it]

Statement: 'record_0' - Result: systematic review
Statement: 'record_1' - Result: systematic review
Statement: 'record_2' - Result: systematic review with meta-analysis
Statement: 'record_3' - Result: systematic review with meta-analysis
Statement: 'record_4' - Result: systematic review with meta-analysis
Statement: 'record_5' - Result: systematic review with meta-analysis
Statement: 'record_6' - Result: systematic review with meta-analysis
Statement: 'record_7' - Result: systematic review with meta-analysis
Statement: 'record_8' - Result: systematic review with meta-analysis
Statement: 'record_9' - Result: systematic review with meta-analysis
Statement: 'record_10' - Result: cluster randomized controlled trial
Statement: 'record_11' - Result: systematic review with meta-analysis
Statement: 'record_12' - Result: cross-sectional study
Statement: 'record_13' - Result: systematic review with meta-analysis
Statement: 'record_14' - Result: systematic review with meta-analysis
Statement: 're

In [64]:
import numpy as np

S_flags = np.array(list(results.items()))[:, 1]

In [65]:
S_df = pd.DataFrame(S_flags, columns=["S"])
S_df = S_df.map(lambda x: x.upper() if isinstance(x, str) else x)

output_file = "S_flags.xlsx"
S_df.to_excel(output_file, index=False)

In [68]:
results_1 = []
for f in S_flags:
    if "trial" in f:
        results_1.append("TRUE")
    else:
        results_1.append("FALSE")

In [73]:
S_df = pd.DataFrame(results_1, columns=["S"])
S_df = S_df.map(lambda x: x.upper() if isinstance(x, str) else x)

output_file = "S_flags.xlsx"
S_df.to_excel(output_file, index=False)